# Identify Pytorch to TensorRT Conversion Path
In my [previos tests](https://github.com/bhlarson/mllib#readme), converting Tensorflow models to TensorRT resulted in a ~ 10x reduction in runtime without reducing inference accuracy.  Following the same process for PyTorch models resulted in an incorrect TensorRT inference.  

[This file](https://github.com/bhlarson/mllib/blob/master/TestONNXTRT.ipynb) isolates the ONNX to TensorRT conversion to identify usage or conversion errors that may explain the TensorRT inference failure.  In troubleshooting this process, I have updated to [TensorRT 8.2](https://docs.nvidia.com/deeplearning/tensorrt/developer-guide/index.html) and incorperated the conversion calls described in TensorRT 8.2 documentation.  

Process:
1. Clone [MLLIB](https://github.com/bhlarson/mllib) project:
```console
git clone https://github.com/bhlarson/mllib.git
```
2. Fom the mllib directory, create an empty creds.json file and, build docker containers:
```console
cd mllib
echo '{}' > creds.json
./build
```
3. Start the NVIDIA pytorch docker container in console mode:
```console
./dtr
```
4. From the docker command line, start the jupyter engine to run this notebook:
```console
./lab
```
5. Open a chrome browser to [http://localhost:9999](http://localhost:9999)
1. Open a TestONNXTRT.ipynb in Jupyter
1. From the Jupyter Lab toolbar, Select Run->Run All Cells to run ONNX inference, ONNX to TensorRT conversion, and TensorRT conversion.
1. Note that ONNX inference segments, people, animals, and vehicles
1. Note the ONNX to TensorRT conversion is successful witout errors
1. Note the TensorRT inference is corrupted with a 4x4 pattern in the inference image which is overlayed on top of the original image.   

The inference results in this example are displayed as a per-chanel mask on top of the original image.  

Of particular note for this study is
1. [onnx-trt.py](./target/onnx-trt.py) implements the ONNX to TRT conversion described in the [NVIDIA TENSORRT DOCUMENTATION](https://docs.nvidia.com/deeplearning/tensorrt/developer-guide/index.html#network_python)
1. [target.trtinference](./target/trtinference.py) implements TRT inference described in the [NVIDIA TENSORRT DOCUMENTATION](https://docs.nvidia.com/deeplearning/tensorrt/developer-guide/index.html#network_python)

In [ ]:
import sys, os
from glob import glob
import tqdm
import onnxruntime as ort
import numpy as np
import cv2
import IPython
# locate imports from either mllib directory or target directory
sys.path.append(os.path.abspath(''))
sys.path.append(os.path.abspath('..'))
from utils.jsonutil import ReadDictJson
from datasets.cocostore import resize_crop_or_pad
from utils.metrics import MergeIman, DatasetResults
from target.trtinference import TrtInference

In [ ]:
def imshow(img):
    import cv2
    import IPython
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

In [ ]:
height = 480
width = 512
batch_size = 1
images = glob('test/*.jpg')
class_dictionary_name= 'test/coco.json'
onnxmodelname='test/segment_nas_512x442_20211119_00.onnx'
trtmodelname='test/segment_nas_512x442_20211119_00.trt'
trtprecision='fp16'

In [ ]:
class_dictionary = ReadDictJson(class_dictionary_name)

In [ ]:
onnxsess = ort.InferenceSession(onnxmodelname)
input_name = onnxsess.get_inputs()[0].name
input_name

In [ ]:
dsResults = DatasetResults(class_dictionary)

In [ ]:
for imagename in images:
    image = cv2.imread(imagename)
    image, imgMean, imgStd = resize_crop_or_pad(image, height,width)
    imageBCHW = np.expand_dims(image.transpose(2, 0, 1),0).astype('float32')
    predonnx = onnxsess.run(None, {input_name: imageBCHW})
    segmentation = np.argmax(predonnx[0], axis=1).astype('uint8')[0]

    iman = MergeIman(image, segmentation, dsResults.lut,imgMean, imgStd)
    imshow(iman)

In [ ]:
!python3 target/onnx-trt-file.py  -onnxname {onnxmodelname} -trtname {trtmodelname} -precision {trtprecision}

In [ ]:
trtserializedmodel = None
with open(trtmodelname, "rb") as f:
    trtserializedmodel = f.read()

In [ ]:
inf = TrtInference(trtserializedmodel, batch_size, height, width, class_dictionary)

In [ ]:
for imagename in images:
    image = cv2.imread(imagename)
    image, imgMean, imgStd = resize_crop_or_pad(image, height,width)
    imageBCHW = np.expand_dims(image.transpose(2, 0, 1),0).astype('float32')
    print('input shape: {}'.format(imageBCHW.shape))
    segmentation = inf.predict(np.ascontiguousarray(imageBCHW))[0]
    iman = MergeIman(image, segmentation, dsResults.lut,imgMean, imgStd)
    imshow(iman)